In [1]:
import pandas as pd
import numpy as np
import neuroCombat as nC
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from scipy.stats import ranksums, ttest_ind, ttest_rel, ks_2samp
import os
import Nestedcombat as nested
from sklearn.preprocessing import StandardScaler
from collections import Counter
#import GMMComBat as gmmc

In [2]:
#Load the data
main_dir= "/home/ulaval.ca/lesee/projects/Project-CNET/"


data = pd.read_csv(os.path.join(main_dir,'data/radiomics_CNET_updated_bin_0_05_batch_clinical.csv'))

# Explicitly create a copy of the data
filtered_data_copy = data.copy()

#drop columns not used in this work
filtered_data_copy.drop(columns = ['Center', 'pixel_spacing', 'StudyInstanceUID', 'Recurrence', 'OS_days', 'PFS_days', 'manufacturer', 'Subtype'], inplace=True)
                        
print("Shape of the dataset with radiomic features : " , np.shape(filtered_data_copy))
#remove a batch level with only one sample


Shape of the dataset with radiomic features :  (95, 1238)


In [3]:
# Store the feature names
feature_names = filtered_data_copy.columns.tolist()

#specify the batch list                                            
batch_list = ['model', 'kernel', 'slice_thickness']
# specify the categorical and continous clinical covariates
categorical_cols = ['Smoking', 'Sex']

continuous_cols = ['Age']
patient_name = filtered_data_copy['PatientName']

# Initialize an empty mask (series of False values)
# This will be used to mark rows to drop
rows_to_drop = pd.Series(False, index=filtered_data_copy.index)

# Loop over each categorical column
for col in batch_list:
    # Calculate the value counts for the current column
    value_counts = filtered_data_copy[col].value_counts()
    
    # Identify values that occur exactly once
    values_to_drop = value_counts[value_counts == 1].index
    
    # Update the mask to include rows where the current column's value is in values_to_drop
    rows_to_drop |= filtered_data_copy[col].isin(values_to_drop)
    print(values_to_drop)

#remove batch levels with only one sample
# Drop rows marked in the mask
filtered_data_copy = filtered_data_copy[~rows_to_drop]
print(np.shape(filtered_data_copy))



Index([], dtype='object')
Index([], dtype='object')
Int64Index([], dtype='int64')
(95, 1238)


In [4]:
#define a string-based covariate datframe                       
covars_string = pd.DataFrame()
covars_string[categorical_cols] = filtered_data_copy[categorical_cols].copy()
covars_string[batch_list] = filtered_data_copy[batch_list].copy()
print("String-based covariate datframe : ", covars_string)


String-based covariate datframe :            Smoking     Sex          model kernel  slice_thickness
0          Smoker    Male     Definition   B40f                1
1          Smoker    Male     Definition   B40f                1
2   Former smoker  Female     Definition   B40f                1
3          Smoker  Female     Definition   B40f                1
4          Smoker  Female     Definition   B40f                1
..            ...     ...            ...    ...              ...
90  Former smoker  Female  SOMATOM Drive  Br43f                2
91  Former smoker  Female  SOMATOM Drive  Br43f                2
92  Former smoker  Female  SOMATOM Drive  Br43f                2
93     Non smoker  Female  SOMATOM Drive  Br43f                2
94  Former smoker  Female  SOMATOM Drive  Br43f                2

[95 rows x 5 columns]


In [5]:
counts_batch_levels = Counter(list(covars_string["slice_thickness"]))

for element, count in sorted(counts_batch_levels.items()):
    print(f"{element}: {count}")

1: 6
2: 89


In [6]:
counts_batch_levels = Counter(list(covars_string["model"]))

for element, count in sorted(counts_batch_levels.items()):
    print(f"{element}: {count}")

Definition: 20
SOMATOM Definition: 70
SOMATOM Drive: 5


In [7]:
counts_batch_levels = Counter(list(covars_string["kernel"]))

for element, count in sorted(counts_batch_levels.items()):
    print(f"{element}: {count}")

B40f: 90
Br43f: 5


In [8]:
#load the continous clinical covariate                       
covars_quant = filtered_data_copy[continuous_cols]

#specify the features
data_df = filtered_data_copy.drop(columns=['PatientName', 'Smoking', 'Age', 'Sex', 'model', 'kernel', 'slice_thickness'])
print(np.sum(np.sum(np.isnan(data_df))))

0


In [10]:
#Remove constant radiomic features
constant_features = data_df.columns[data_df.nunique() == 1]
data_df.drop(constant_features, axis=1, inplace=True)
data_df = data_df.reset_index(drop=True)
print(np.shape(data_df))

(95, 1230)


In [12]:
dat = data_df.T.apply(pd.to_numeric)                    
#print(dat)
#label-encode the string covariates
covars_cat = pd.DataFrame()
for col in covars_string:
    stringcol = covars_string[col]
    le = LabelEncoder()
    le.fit(list(stringcol))
    covars_cat[col] = le.transform(stringcol)


In [15]:
covars_cat_reset = covars_cat.reset_index(drop=True)
covars_quant_reset = covars_quant.reset_index(drop=True)
#concatenate the label-enoded categorical (batch+clinical) and continous clinical covariates                         
covars = pd.concat([covars_cat_reset, covars_quant_reset], axis=1)
print(categorical_cols)
print(continuous_cols)
# Check if there are any NaN values in covars and batch_col
covars_nan_count = np.sum(np.sum(np.isnan(data_df)))
print("Number of NaN values in covars:", covars_nan_count)
#print("Number of NaN values in batch_col:", batch_col_nan_count)

['Smoking', 'Sex']
['Age']
Number of NaN values in covars: 0


In [55]:
print(np.shape(dat))
#Nested Combat
output_data = nested.NestedComBat(dat, covars, batch_list, categorical_cols=categorical_cols,
                                   continuous_cols=continuous_cols, drop=True, write_p=False, filepath=main_dir)
output_data_reset = output_data.reset_index(drop=True)
patient_name_reset = patient_name.reset_index(drop=True)
write_df = pd.concat([patient_name_reset, output_data_reset], axis=1)
write_df.to_csv(os.path.join(main_dir,'results/data_combat/combat_harmonized_features_nested_cnet_updated_bw_0.05_D.csv'), float_format='%.7f', index= False)

(1230, 95)
ROUND 1:
Harmonizing by model...
ComBat with Raw Data...
[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data
Harmonizing by kernel...
ComBat with Raw Data...
[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data
Harmonizing by slice_thickness...
ComBat with Raw Data...
[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data
ROUND 2:
Harmonizing by model...
[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding